In [1]:
import Model as md
import Resnet as res
import data as dta
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_metric_learning import losses
from torchvision import transforms as T
from torch.utils.data import DataLoader
from absl import app
from absl import flags
from absl import logging
import os
from tqdm import tqdm

from torch.cuda import amp
import pandas as pd
import math

In [2]:
encoder = res.ResNet50().cuda()
proj_head = md.Projection_Head(encoder.representation_dim, 1).cuda()
sup_head = md.Supervised_Head(proj_head.output, 5).cuda()

In [3]:
transf_eval = T.Compose([
    T.CenterCrop(380),
    T.Resize(112),
    T.ToTensor(),
])

In [4]:
astro_ds = dta.AstroDataset('nairTest.csv', 'imagenes_clasificadas_nair/', transform=transf_eval)
dataset_astro = DataLoader(astro_ds,batch_size=128, shuffle=False,num_workers=6)

In [5]:
checkpoint = torch.load('finetuned_model/model.pt')
encoder.load_state_dict(checkpoint['encoder'])
proj_head.load_state_dict(checkpoint['projection_head'], strict=False)
sup_head.load_state_dict(checkpoint['supervised_head'])

<All keys matched successfully>

In [6]:
encoder.eval()
proj_head.eval()
sup_head.eval()

Supervised_Head(
  (supervised_head): Linear(in_features=2048, out_features=5, bias=True)
)

In [7]:
softmax = nn.Softmax(dim=1)

In [8]:
tqdm_loop = tqdm(enumerate(dataset_astro), total=len(dataset_astro), leave=True)
acc = 0
for batch_idx, data in tqdm_loop:
    with torch.no_grad():
        image = data[0].cuda()
        labels = data[1].cuda()
        output_val = sup_head(proj_head(encoder(image)))
        output_val = softmax(output_val)
        acc_val = torch.argmax(output_val,dim=1)
        acc_val = (acc_val == labels).float().sum()
        acc += acc_val
acc /= astro_ds.__len__()
print('Acc {}'.format(acc*100))

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:07<00:00,  4.14it/s]

Acc 63.99203109741211


In [9]:
astro_ds.__len__()

4013